#<center>**Coleta de Dados de CNPJ das empresas listadas na B3**</center>
---
Coleta de dados de CNPJ de todas as empresas e fundos imobiliários listados na bolsa de valores brasileira (B3) do site Infomoney.

Projeto desenvolvido por Dimas Tadeu Parreiras





# **Importando as Bibliotecas**

In [1]:
import pandas as pd
import numpy as np
import requests

# **Coleta e Tratamento de Dados**
Acessando o site para coletar as informações sobre cada ativo

 > ***Site Ações: https://www.infomoney.com.br/minhas-financas/confira-o-cnpj-das-acoes-negociadas-em-bolsa-e-saiba-como-declarar-no-imposto-de-renda/***

> ***Site FIIs: https://www.infomoney.com.br/minhas-financas/confira-o-cnpj-dos-fundos-imobiliarios-listados-e-saiba-como-declarar-no-imposto-de-renda/***

## **Função - Define_URL**


> Função para preencher a URL do site com a ação a ser pesquisada

In [2]:
# Informações para fingir ser um navegador
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

In [3]:
def Define_URL(site):
  # URL do site
  url = site
  # Juntando a URL com as informações de navegador
  url = requests.get(url, headers=header)
  return url

## **Função - Ler_URL**


> Função para ler e importar os dados do site

In [4]:
def Ler_URL(url):
  # Lendo as informações do site Fundamentus
  acao = pd.read_html(url.text)
  return acao

## **Web Scrapping**




In [5]:
# DataFrame que vai guardar todos os dados
df_cnpj = pd.DataFrame(columns=['CÓDIGO', 'RAZÃO SOCIAL', 'NOME DE PREGÃO', 'CNPJ', 'TIPO'])

### **Ações**

In [6]:
# Coletando dos dados
url = Define_URL('https://www.infomoney.com.br/minhas-financas/confira-o-cnpj-das-acoes-negociadas-em-bolsa-e-saiba-como-declarar-no-imposto-de-renda/')
info = Ler_URL(url)
info = info[0]

In [7]:
display(info)

,Nome de Pregão,Razão Social,CNPJ,Código
0,ADVANCED-DH,ADVANCED DIGITAL HEALTH MEDICINA PREVENTIVA S.A.,10.345.009/0001-98,"ADHM1, ADHM3"
1,AES TIETE E,AES TIETE ENERGIA SA,04.128.563/0001-10,"TIET11, TIET3, TIET4"
2,AFLUENTE T,AFLUENTE TRANSMISSÃO DE ENERGIA ELÉTRICA S/A,10.338.320/0001-00,AFLT3
3,ALEF S/A,ALEF S.A.,02.217.319/0001-07,ALEF3B
4,ALFA HOLDING,ALFA HOLDINGS S.A.,17.167.396/0001-69,"RPAD3, RPAD5, RPAD6"
...,...,...,...,...
416,WILSON SONS,WILSON SONS LTD.,05.721.735/0001-28,WSON33
417,WIZ S.A.,WIZ SOLUÇÕES E CORRETAGEM DE SEGUROS S.A.,42.278.473/0001-03,WIZS3
418,WLM IND COM,WLM PART. E COMÉRCIO DE MÁQUINAS E VEÍCULOS S.A.,33.228.024/0001-51,"WLMM3, WLMM4"
419,YDUQS PART,YDUQS PARTICIPACOES S.A.,08.807.432/0001-10,YDUQ3


In [8]:
# Quebrando os dados para separar linhas com mais de um ticker
for i in range(len(info)):
  ticker = info.iloc[i]['Código'].split(', ')
  if len(ticker) > 1:
    for t in ticker:
      linha = []
      linha.append(t)
      linha.append(info.iloc[i]['Razão Social'])
      linha.append(info.iloc[i]['Nome de Pregão'])
      linha.append(info.iloc[i]['CNPJ'])
      linha.append('AÇÃO')
      df_cnpj = df_cnpj.append(pd.DataFrame(data=[[linha[0], linha[1], linha[2], linha[3], linha[4]]], columns=['CÓDIGO', 'RAZÃO SOCIAL', 'NOME DE PREGÃO', 'CNPJ', 'TIPO']))
  else:
    linha = []
    linha.append(ticker[0])
    linha.append(info.iloc[i]['Razão Social'])
    linha.append(info.iloc[i]['Nome de Pregão'])
    linha.append(info.iloc[i]['CNPJ'])
    linha.append('AÇÃO')
    df_cnpj = df_cnpj.append(pd.DataFrame(data=[[linha[0], linha[1], linha[2], linha[3], linha[4]]], columns=['CÓDIGO', 'RAZÃO SOCIAL', 'NOME DE PREGÃO', 'CNPJ', 'TIPO']))
  

In [9]:
# Resetando o index
df_cnpj.reset_index(inplace=True, drop=True)

In [10]:
display(df_cnpj)

,CÓDIGO,RAZÃO SOCIAL,NOME DE PREGÃO,CNPJ,TIPO
0,ADHM1,ADVANCED DIGITAL HEALTH MEDICINA PREVENTIVA S.A.,ADVANCED-DH,10.345.009/0001-98,AÇÃO
1,ADHM3,ADVANCED DIGITAL HEALTH MEDICINA PREVENTIVA S.A.,ADVANCED-DH,10.345.009/0001-98,AÇÃO
2,TIET11,AES TIETE ENERGIA SA,AES TIETE E,04.128.563/0001-10,AÇÃO
3,TIET3,AES TIETE ENERGIA SA,AES TIETE E,04.128.563/0001-10,AÇÃO
4,TIET4,AES TIETE ENERGIA SA,AES TIETE E,04.128.563/0001-10,AÇÃO
...,...,...,...,...,...
594,WIZS3,WIZ SOLUÇÕES E CORRETAGEM DE SEGUROS S.A.,WIZ S.A.,42.278.473/0001-03,AÇÃO
595,WLMM3,WLM PART. E COMÉRCIO DE MÁQUINAS E VEÍCULOS S.A.,WLM IND COM,33.228.024/0001-51,AÇÃO
596,WLMM4,WLM PART. E COMÉRCIO DE MÁQUINAS E VEÍCULOS S.A.,WLM IND COM,33.228.024/0001-51,AÇÃO
597,YDUQ3,YDUQS PARTICIPACOES S.A.,YDUQS PART,08.807.432/0001-10,AÇÃO


### **FIIs**

In [11]:
# Coletando dos dados
url = Define_URL('https://www.infomoney.com.br/minhas-financas/confira-o-cnpj-dos-fundos-imobiliarios-listados-e-saiba-como-declarar-no-imposto-de-renda/')
info = Ler_URL(url)
info = info[0]

In [12]:
info.rename(columns={'FUNDO' : 'NOME DE PREGÃO'}, inplace=True)
info['TIPO'] = 'FII'

In [13]:
display(info)

,NOME DE PREGÃO,CÓDIGO,RAZÃO SOCIAL,CNPJ,TIPO
0,FII A BRANCA,FPAB11,FDO INV IMOB PROJETO ÁGUA BRANCA,03.251.720/0001-18,FII
1,FII ABC IMOB,ABCP11,FDO INV IMOB GRAND PLAZA SHOPPING,01.201.140/0001-90,FII
2,FII ABSOLUTO,BPFF11,FDO INV IMOB BRASIL PLURAL ABSOLUTO FDO DE FUNDOS,17.324.357/0001-28,FII
3,FII AFINVCR,AFCR11,AF INVEST FDO INV. IMOB. – RECEBÍVEIS IMOB.,32.065.364/0001-46,FII
4,FII ALIANZA,ALZR11,ALIANZA TRUST RENDA IMOBILIARIA FDO INV IMOB,28.737.771/0001-85,FII
...,...,...,...,...,...
271,FII XP MACAE,XPCM11,XP CORPORATE MACAÉ FDO INV IMOB – FII,16.802.320/0001-03,FII
272,FII XP MALLS,XPML11,XP MALLS FDO INV IMOB FII,28.757.546/0001-00,FII
273,FII XP PROP,XPPR11,XP PROPERTIES FDO. INV. IMOB.,30.654.849/0001-40,FII
274,FII XP SELEC,XPSF11,XP SELECTION FDO DE FUNDOS INV IMOB – FII,30.983.020/0001-90,FII


In [14]:
df_cnpj = pd.concat([df_cnpj, info],
                         axis = 0,
                         join = 'outer')

In [15]:
df_cnpj.reset_index(inplace=True, drop=True)

In [16]:
display(df_cnpj)

,CÓDIGO,RAZÃO SOCIAL,NOME DE PREGÃO,CNPJ,TIPO
0,ADHM1,ADVANCED DIGITAL HEALTH MEDICINA PREVENTIVA S.A.,ADVANCED-DH,10.345.009/0001-98,AÇÃO
1,ADHM3,ADVANCED DIGITAL HEALTH MEDICINA PREVENTIVA S.A.,ADVANCED-DH,10.345.009/0001-98,AÇÃO
2,TIET11,AES TIETE ENERGIA SA,AES TIETE E,04.128.563/0001-10,AÇÃO
3,TIET3,AES TIETE ENERGIA SA,AES TIETE E,04.128.563/0001-10,AÇÃO
4,TIET4,AES TIETE ENERGIA SA,AES TIETE E,04.128.563/0001-10,AÇÃO
...,...,...,...,...,...
870,XPCM11,XP CORPORATE MACAÉ FDO INV IMOB – FII,FII XP MACAE,16.802.320/0001-03,FII
871,XPML11,XP MALLS FDO INV IMOB FII,FII XP MALLS,28.757.546/0001-00,FII
872,XPPR11,XP PROPERTIES FDO. INV. IMOB.,FII XP PROP,30.654.849/0001-40,FII
873,XPSF11,XP SELECTION FDO DE FUNDOS INV IMOB – FII,FII XP SELEC,30.983.020/0001-90,FII


In [18]:
df_cnpj.to_csv('/content/drive/MyDrive/Colab Notebooks/Projetos_Mercado_Financeiro/Controle_de_Carteira/Dados_CNPJ_Empresas.csv', index=False)